# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [46]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler,LabelEncoder
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score


### 2. Load the dataset

In [2]:
df=pd.read_csv("car+data.csv")
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [5]:
df.shape

(301, 9)

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [6]:
len(df[df.duplicated()])

2

In [8]:
df.drop_duplicates(keep='first',inplace=True)

In [9]:
len(df[df.duplicated()])

0

### 5. Drop the columns which you think redundant for the analysis.

In [14]:
df.drop('Car_Name',axis=1,inplace=True)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [28]:
from datetime import date
today=date.today().year
df['age_of_the_car']=today-df['Year']
df['age_of_the_car']

0       8
1       9
2       5
3      11
4       8
       ..
296     6
297     7
298    13
299     5
300     6
Name: age_of_the_car, Length: 299, dtype: int64

In [30]:
df.drop('Year',axis=1,inplace=True)

### 7. Encode the categorical columns

In [34]:
for col in df.select_dtypes('object').columns:
    le=LabelEncoder()
    df[col]=df[col].apply(le.fit_transform)


df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,2,0,1,0,8
1,4.75,9.54,43000,1,0,1,0,9
2,7.25,9.85,6900,2,0,1,0,5
3,2.85,4.15,5200,2,0,1,0,11
4,4.60,6.87,42450,1,0,1,0,8


### 8. Separate the target and independent features.

In [36]:
X=df.drop('Selling_Price',axis=1)
Y=df['Selling_Price']

### 9. Split the data into train and test.

In [38]:
## Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.30,random_state=1)
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [48]:
def fit_n_predict(model,X_train,X_test,Y_train,Y_test):
    model.fit(X_train,Y_train)    
    pred=model.predict(X_test)
    accuracy=r2_score(Y_test,pred)
    return accuracy

In [50]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()

In [51]:
rs=pd.DataFrame()

In [52]:
result=fit_n_predict(rf,X_train,X_test,Y_train,Y_test)
result

0.9113272505620086

In [53]:
rs['random_forest']=pd.Series(result)
rs

,random_forest
0,0.911327


### 11. Create a pickle file with an extension as .pkl

In [ ]:
import pickle
#Saving model to disk
pickle.dump(rf,open('model.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <div class="hero-image">
        <div class="hero-text">
            <h1 style="font-size:50px">Used Car Price Predictor</h1>
            <br><br><h3>{{prediction_text}}</h3>
        </div>
    </div>
    <style>
        body,html{
            height:100%;
            margin:0;
            font-family:Arial,Helvetica,sans-serif;
        }
        .hero-image{
            background-image: linear-gradient(rgba(0,0,0,0.5),rgba(0,0,0,0.5)),url('/static/image.jpg');
            height: 50%;
            background-position: bottom;
            background-repeat: no-repeat;
            background-size: cover;
        }
        .hero-text{
            text-align: center;
            position: absolute;
            top:50%;
            left:50%;
            transform: translate(-50%,-50%);
            color: white;
        }
    </style>
    <div style="color: rgb(0,0,0)">
    <form action="{{url_for('predict')}}" method="post">
        <h2>Enter Car Details:</h2>
        <h3>Age of the car(in years)</h3>
        <input id="First" name="Age_of_the_car" type="number">
        <h3>Present Showroom price(In lakhs)</h3><br><input id="second" name="present_price" required="required">
        <h3>Kilometers Driven</h3><input id="third" name="Kms_Driven" required="required">
        <h3>Owner Type(0/1/2)</h3><br><input id="fourth" name="Owner" required="required">
        <h3>Fuel type</h3><br><select name="Fuel_Type" id="fuel" required="required">
            <option value="0">Petrol</option>
            <option value="1">Diesel</option>
            <option value="2">CNG</option>
        </select>
        <h3>Seller Type</h3><br><select name="seller_Type" id="resea" required="required">
            <option value="0">Dealer</option>
            <option value="1">Individual</option>
        </select>
        <h3>Transmission type</h3><br><select name="Transmission" id="research" required="required">
            <option value="0">Manual Car</option>
            <option value="1">Automatic Car</option>
             
        </select>
        <br><br><button id="sub" type="submit">Predict selling price</button>

        
    </form>
    </div>
    <style>
        body{
            background-color: 101,10,20;
            text-align: center;
            padding:0px;
            font-family: Helvetica;
        }
        #research{
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }
        #box{
            border-radius:60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }
        #fuel{
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }
        #fuel:hover{
            background-color: white;
        }
        #research{
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }
        #research:hover{
            background-color: white;
        }
        #resea{
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }
        #resea:hover{
            background-color: white;
        }
        #sub{
            background-color: green;
            font-family:'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align:center;
            border-radius:20px;
            font-size: 18px;
            color: white;
        }
        #sub:hover{
            background-color: white;
        }
        #first{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #second{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #third{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        #fourth{
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
        

    </style>
    
</body>
</html>

### b) Create app.py file and write the predict function

In [ ]:
# importing necessary libraries and functions
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__) #Initialize the flask App
model = pickle.load(open('model.pkl', 'rb')) # loading the trained model

@app.route('/') # Homepage
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    
    # retrieving values from form
    init_features = [float(x) for x in request.form.values()]

    final_features = [np.array(init_features)]
    prediction = model.predict(final_features) # making prediction


    return render_template('index.html', prediction_text='Predicted Class: {}'.format(prediction)) # rendering the predicted result

if __name__ == "__main__":
    app.run(debug=True)

### 13. Deploy your app on Heroku. (write commands for deployment)

In [ ]:
Follow the below steps to deploy your application on heroku
First We need login to heroku and create a heroku app.

heroku login (then login to heroku with your the id and password).
Create a heroku app

heroku create nameofyourapp (go back to heroku dashboard we can see a app created with given name)
Create a Procfile (Procfile is a mechanism for declaring what commands are run by your application’s dynos on the Heroku platform.)

echo web: >Procfile (creates a procfile)
web: gunicorn app:app(paste this in the procfile)
Create a runtime.txt file (need to tell what type of python need to be used)

python-3.10.5 (pass your version of the python in the file)
Create a requirements.txt file

pip freeze >requirements.txt
initialize a git repository git init

Push your code to git repository

git add .
git commit -m 'Intial Commit'
Push the code from your local repository's main branch to your heroku remote.

heroku git:remote -a nameofyourapp (this will set git remote heroku)
git push heroku master

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

### Happy Learning :)